# Day 1 - Lab 1: AI-Powered Requirements & User Stories

**Objective:** Use a Large Language Model (LLM) to decompose a vague problem statement into structured features, user personas, and Agile user stories, culminating in a machine-readable JSON artifact.

**Estimated Time:** 90 minutes

**Introduction:**
Welcome to the first hands-on lab of the AI-Driven Software Engineering Program! All great software projects begin with a clear understanding of the problem to be solved. In this lab, you will take on the role of a tech lead or product manager and use an LLM as a co-pilot to transform a simple, high-level problem into a set of well-defined, actionable requirements. This process is fundamental to ensuring that the team builds the *right* product.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM).

**Model Selection:**
Our `utils.py` script is configured to work with multiple AI providers. You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in the `RECOMMENDED_MODELS` dictionary in `utils.py`. For example, to use a Hugging Face model, you could change the line to: `client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-3.3-70B-Instruct")`

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client for our chosen LLM.
- `get_completion()`: To send a prompt to the LLM and get a response.
- `save_artifact()`: To save our generated requirements to a file.

In [1]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, prompt_enhancer

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

2025-10-28 13:14:44,712 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Problem Statement

Every project starts with a problem. Our problem is a common one in many organizations:

> **"We need a tool to help our company's new hires get up to speed."**

This statement is intentionally vague. Our job is to use the LLM to add structure and detail to it.

In [2]:
problem_statement = "We need a tool to help our company's new hires get up to speed."

## Step 3: The Challenges

Complete the following challenges in order. Each one builds upon the last, increasing in technical complexity and value.

### Challenge 1 (Foundational): Brainstorming Features

**Task:** Use the LLM to brainstorm a list of potential features and user personas based on the problem statement.

**Instructions:**
1. Write a simple prompt that asks the LLM to brainstorm features for the onboarding tool.
2. Write a second prompt to identify three distinct user personas who would use this tool.
3. Run both prompts and review the markdown output.

**Expected Quality:** The output should be a simple, readable markdown list of features and a description of the personas. This is a good first step but lacks the structure needed for automation.

In [3]:
# TODO: Create a string variable named 'features_prompt'.
# This prompt should ask the LLM to brainstorm features based on the problem_statement.
features_prompt =  f"Brainstorm a list of features based on: {problem_statement}"
enhanced_features_prompt = prompt_enhancer(features_prompt)

print("--- Enhanced Features Prompt ---")
print(enhanced_features_prompt)

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(enhanced_features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# TODO: Create a string variable named 'personas_prompt'.
# This prompt should ask the LLM to identify three user personas based on the problem_statement.
personas_prompt = f"Identify three user personas based on : {problem_statement}"
enhanced_personas_prompt = prompt_enhancer(personas_prompt)

print("\n--- Identifying User Personas ---")
user_personas = get_completion(enhanced_personas_prompt, client, model_name, api_provider)
print(user_personas)

2025-10-28 13:14:45,201 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Enhanced Features Prompt ---
<prompt>
    <persona>
        You are a seasoned HR-tech Product Strategist with 15+ years of experience designing SaaS onboarding platforms for Fortune 500 companies. You specialize in translating employee-experience pain points into high-impact, feasible product features.
    </persona>

    <context>
        Goal: Brainstorm product features for a new internal tool that accelerates our company’s new-hire ramp-up.
        Company profile: 3,000-person global tech firm, hybrid workforce, multiple departments (Engineering, Sales, Customer Success, Operations).
        Success criteria:
            • Reduce average time to productivity (TTP) in role by ≥25 %.
            • Improve new-hire satisfaction scores in first-90-day survey.
            • Minimize burden on managers and HR through automation and self-service.
        Constraints:
            • Must comply with SOC 2 and GDPR.
            • Integrate with existing systems: Google Workspace, Slack

2025-10-28 13:15:36,348 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Of course. As a seasoned HR-tech strategist, I understand the critical importance of a structured, supportive, and efficient onboarding experience. A well-designed internal tool isn't just about checklists; it's about accelerating connection, competence, and confidence.

Here is a breakdown of potential features for your new-hire ramp-up tool, organized by the employee journey and prioritized by business value.

***

## Pre-boarding (Offer Signed → Day 1)

This phase is crucial for mitigating buyer's remorse and building excitement. The goal is to handle logistics seamlessly and begin cultural immersion before the new hire even walks through the door (virtual or physical).

*   **Feature Name:** Onboarding Journey Orchestrator
    *   **Description:** An automated, role-based workflow engine that assigns and tracks pre-boarding tasks for HR, IT, and the hiring manager (e.g., "Order laptop," "Assign buddy," "Add to Google Groups"). Integrates with Jira to create tickets automatically.
 

### Challenge 2 (Intermediate): Generating Formal User Stories

**Task:** Now, let's increase the value by generating structured, formal Agile User Stories.

**Instructions:**
1. Create a new, more sophisticated prompt.
2. This prompt should instruct the LLM to act as a Senior Product Manager.
3. It must use the brainstormed features and personas from the previous step as context.
4. The key instruction is to generate a list of user stories, each with detailed acceptance criteria in Gherkin format (`Given/When/Then`).
5. **Crucially, the prompt must demand the final output be a well-formed JSON array of objects.** Each object should represent a user story and have keys like `id`, `user_story`, `persona`, and `acceptance_criteria`.

> **Tip:** If the LLM's output isn't perfect JSON, try making your prompt even more specific. You can tell it, 'Do not include any text before or after the JSON array. Your response must begin with [ and end with ].'

**Expected Quality:** The output should not be markdown, but a clean, parsable JSON string. This is a significant step up in value, as a JSON artifact can be automatically processed by other systems (e.g., imported into Jira).

In [4]:
# TODO: Create a detailed prompt string named 'json_user_stories_prompt'.
# This prompt needs to instruct the LLM to act as a Senior Product Manager and convert the
# brainstormed features and personas into a structured JSON array of user stories.
# Tip: Be very specific about the required JSON format in your prompt instructions. Tell it what keys to use and what the data types should be.
json_user_stories_prompt = f"""

You are a Senior Product Manager. Your task is to create user stories in JSON format.

 

Here is the context:

Problem statement: "{problem_statement}"

 

Brainstormed features:

{brainstormed_features}

 

User personas:

{user_personas}

 

Using this information, produce a structured JSON array of user stories. Each user story must follow this schema:

- "id": integer (unique, starting from 1)

- "persona": string (the persona the story is for)

- "feature": string (the related feature)

- "user_story": string (formatted as "As a [persona], I want [feature] so that [benefit].")

- "acceptance_criteria": list of 2–3 short strings

- "priority": string ("High", "Medium", or "Low")

 

Return **only** valid JSON. Do not include explanations, markdown, or commentary.

Example format:

[

  {{

    "id": 1,

    "persona": "Ava, New Software Engineer",

    "feature": "Interactive onboarding checklist",

    "user_story": "As a new software engineer, I want an interactive onboarding checklist so that I can track my progress easily.",

    "acceptance_criteria": [

      "Checklist items can be marked complete",

      "Progress is automatically saved"

    ],

    "priority": "High"

  }}

]

"""

print("--- Generating User Stories as JSON ---")
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Let's try to parse the JSON to see if the LLM followed instructions
try:
    # The LLM might wrap the JSON in markdown fences (```json ... ```).
    # We'll clean that up before parsing.
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    
    if user_stories_json:
        print("\n--- Sample User Story ---")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = []

--- Generating User Stories as JSON ---
Successfully parsed LLM output as JSON.

--- Sample User Story ---
{
  "id": 1,
  "persona": "Maria the Hybrid Manager",
  "feature": "Onboarding Journey Orchestrator",
  "user_story": "As a new manager, I want an automated workflow to assign and track pre-boarding tasks so that I can ensure my new hire has everything they need on Day 1 without manual follow-up.",
  "acceptance_criteria": [
    "When a new hire is added, tasks are automatically created in Jira for IT, HR, and the hiring manager.",
    "I can view the status (To Do, In Progress, Done) of all pre-boarding tasks for my new hire.",
    "The system sends reminders for overdue tasks."
  ],
  "priority": "High"
}


### Challenge 3 (Advanced): Programmatic Validation and Artifact Creation

**Task:** Now for the highest-value step. Instead of just looking at the JSON, we will programmatically validate it and save it as a formal project artifact. This ensures reliability and prepares the requirements for automated use in later stages of the SDLC.

**Instructions:**
1. Complete the `validate_and_save_stories` function below.
2. The function should iterate through the list of stories.
3. For each story, it must validate that the required keys are present and that the acceptance criteria list is not empty.
4. If all stories are valid, it should save the data to `artifacts/day1_user_stories.json`.

**Expected Quality:** A robust script that guarantees the integrity of our requirements artifact. The final output is a validated `day1_user_stories.json` file in the `artifacts` directory, ready to be used as a reliable input for Day 2.

In [5]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    # TODO: Implement the validation logic inside this function.
    # 1. Loop through each story in the 'stories_data' list.
    # 2. For each story, check if it contains all the 'required_keys'.
    # 3. Also check if the 'acceptance_criteria' list is not empty.
    # 4. If a story is invalid, print an error message and set 'all_stories_valid' to False.
    #    (You can use 'continue' to skip to the next story).

     # Validate each story in the list
    for story in stories_data:
        story_id = story.get("id", "UNKNOWN")

        # 1. Check required keys exist
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"[Story {story_id}] Missing keys: {missing_keys}")
            all_stories_valid = False
            continue  # skip deeper checks for this story

        # 2. Check acceptance_criteria exists, is a non-empty list
        ac = story.get("acceptance_criteria")
        if not isinstance(ac, list) or not ac:
            print(f"[Story {story_id}] Invalid acceptance_criteria. "
                  f"Expected a non-empty list, got: {ac!r}")
            all_stories_valid = False
            continue

        # 3. (Optional sanity checks, still counts as valid unless extreme)
        if not story["user_story"] or not isinstance(story["user_story"], str):
            print(f"[Story {story_id}] user_story must be a non-empty string.")
            all_stories_valid = False
            continue

        if not story["persona"] or not isinstance(story["persona"], str):
            print(f"[Story {story_id}] persona must be a non-empty string.")
            all_stories_valid = False
            continue

    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/day1_user_stories.json"
        
        # Convert the validated Python object to a JSON string
        stories_json_str = json.dumps(stories_data, indent=4)

        # Save the serialized JSON using your utility function
        save_artifact(stories_json_str, artifact_path)
        
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation on the JSON data from the previous step
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or not defined.")


All user stories passed validation.


## Lab Conclusion

Congratulations! You have completed the first lab. You started with a vague, one-sentence problem and finished with a structured, validated, machine-readable requirements artifact. This is the critical first step in an AI-assisted software development lifecycle. The `day1_user_stories.json` file you created will be the direct input for our next lab, where we will generate a formal Product Requirements Document (PRD).

> **Key Takeaway:** The single most important skill demonstrated in this lab is turning unstructured ideas into structured, machine-readable data (JSON). This transformation is what enables automation and integration with other tools later in the SDLC.